In [14]:
import pandas as pd
import pycountry

In [70]:
df = pd.read_csv("code/Brgtt07/data/country/GlobalWeatherRepository.csv")
df.head()

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1715849100,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1,1,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1715849100,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1,1,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1715849100,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1,1,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1715849100,2024-05-16 10:45,6.3,43.3,Light drizzle,...,0.7,0.9,1,1,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1715849100,2024-05-16 09:45,26.0,78.8,Partly cloudy,...,183.4,262.3,5,10,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55


In [8]:
df["country"].nunique()

210

In [6]:
df.shape

(59633, 41)

In [9]:
# get the conutries and the temperature

df_selected = df[['country','temperature_celsius']]

In [10]:
df_selected.head()

,country,temperature_celsius
0,Afghanistan,26.6
1,Albania,19.0
2,Algeria,23.0
3,Andorra,6.3
4,Angola,26.0


In [43]:
# get the countries with english name 

def get_english_country_name(country_name):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.name
    except LookupError:
        return None
    

In [44]:
df_selected["english_country"] = df_selected["country"].apply(get_english_country_name)

/tmp/ipykernel_20980/4186824219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["english_country"] = df_selected["country"].apply(get_english_country_name)


In [25]:
df_filtered_name = df_selected.dropna(subset=["english_country"])

In [45]:
# correct the wrong/different country names

countries_correction = {
    "Bolivia, Plurinational State of": "Bolivia",
    "Brunei Darussalam": "Brunei",
    "Congo": "Democratic Republic of the Congo",
    "Czechia": "Czech Republic",
    "Iran, Islamic Republic of": "Iran",
    "Korea, Democratic People's Republic of": "North Korea",
    "Korea, Republic of": "South Korea",
    "Lao People's Democratic Republic": "Laos",
    "Moldova, Republic of": "Moldova",
    "Russian Federation": "Russia",
    "Sao Tome and Principe": "Sao Tome and Principe",
    "Syrian Arab Republic": "Syria",
    "Taiwan, Province of China": "Taiwan",
    "United Kingdom": "United Kingdom of Great Britain and Northern Ireland",
    "Venezuela, Bolivarian Republic of": "Venezuela",
    "Viet Nam": "Vietnam"
}

In [46]:
# apply the correction

df_filtered_name["english_country"] = df_filtered_name["english_country"].replace(countries_correction)

/tmp/ipykernel_20980/2756168295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_name["english_country"] = df_filtered_name["english_country"].replace(countries_correction)


In [49]:
df_final = df_filtered_name.drop(columns=["country"])

In [55]:
df_final['english_country'].value_counts()

Bulgaria     646
Indonesia    615
Sudan        612
Iran         612
Thailand     611
            ... 
Guatemala    302
Colombia       1
Laos           1
Libya          1
Togo           1
Name: english_country, Length: 178, dtype: int64

In [60]:
# get avarage temperature for each country

df_grouped = df_final.groupby("english_country", as_index= False)["temperature_celsius"].mean().round(1)

In [64]:
df_grouped = df_grouped.rename(columns= {"english_country": "Country", "temperature_celsius": "Temperature"})

In [67]:
df_grouped.head()

,Country,Temperature
0,Afghanistan,18.1
1,Albania,22.2
2,Algeria,22.8
3,Andorra,11.1
4,Angola,27.1


In [68]:
# save the clean dataset

df_grouped.to_csv("climate_avarage_temperature.csv", index=False)